In [ ]:
# 1. 安裝必要套件
!pip install streamlit torch torchvision grad-cam opencv-python-headless -q

import os
# 檢查資料夾是否存在
if not os.path.exists('dataset'):
    print("⚠️ 警告：找不到 'dataset' 資料夾！")
    print("請在左側檔案區建立 'dataset' 資料夾，並確認結構如下：")
    print("dataset/train/raccoon (放入浣熊圖)")
    print("dataset/train/tanuki  (放入狸貓圖)")
    print("dataset/train/other   (放入其他圖)")
else:
    print("✅ 環境與資料夾檢查完畢，請繼續下一步。")

✅ 環境與資料夾檢查完畢，請繼續下一步。


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import os
import shutil # Import shutil for directory removal

def train_pipeline():
    DATA_DIR = 'dataset'
    MODEL_SAVE_PATH = 'raccoon_tanuki_model.pth'

    if not os.path.exists(DATA_DIR):
        print("❌ 錯誤：沒有資料，無法訓練。請先上傳圖片。")
        return

    # 清理 .ipynb_checkpoints 資料夾，避免 ImageFolder 誤判為類別
    for phase in ['train', 'val']:
        checkpoints_path = os.path.join(DATA_DIR, phase, '.ipynb_checkpoints')
        if os.path.exists(checkpoints_path) and os.path.isdir(checkpoints_path):
            print(f"🗑️ 偵測到並移除無效的資料夾: {checkpoints_path}")
            shutil.rmtree(checkpoints_path)

    # 設定裝置
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"🚀 使用裝置進行訓練: {device}")

    # 影像預處理
    data_transforms = {
        'train': transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(), # 資料增強
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        # 驗證集若沒建，這裡會自動fallback只用訓練集，方便測試
        'val': transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }

    # 讀取資料
    # 如果只有 train 資料夾，就只讀 train
    phases = ['train']
    if os.path.exists(os.path.join(DATA_DIR, 'val')):
        phases.append('val')

    image_datasets = {x: datasets.ImageFolder(os.path.join(DATA_DIR, x), data_transforms[x]) for x in phases}
    dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True) for x in phases}
    class_names = image_datasets['train'].classes

    print(f"📂 偵測到的類別: {class_names}")

    # 載入 ResNet18
    model = models.resnet18(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, len(class_names))
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    # 訓練迴圈 (設為 5 epoch 快速測試，想要準一點可改成 10-15)
    epochs = 5
    print(f"💪 開始訓練 (共 {epochs} 輪)...")

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders['train']:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(image_datasets['train'])
        epoch_acc = running_corrects.double() / len(image_datasets['train'])
        print(f'Epoch {epoch+1}/{epochs} - Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    # 儲存模型
    torch.save({'model_state_dict': model.state_dict(), 'classes': class_names}, MODEL_SAVE_PATH)
    print(f"✅ 模型已儲存為: {MODEL_SAVE_PATH}")

# 執行訓練
train_pipeline()

🗑️ 偵測到並移除無效的資料夾: dataset/val/.ipynb_checkpoints
🚀 使用裝置進行訓練: cpu
📂 偵測到的類別: ['other', 'raccoon', 'tanuki']


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 166MB/s]


💪 開始訓練 (共 5 輪)...
Epoch 1/5 - Loss: 0.9280 Acc: 0.5556
Epoch 2/5 - Loss: 0.4502 Acc: 0.8889
Epoch 3/5 - Loss: 0.2674 Acc: 0.9259
Epoch 4/5 - Loss: 0.1880 Acc: 0.9605
Epoch 5/5 - Loss: 0.1302 Acc: 0.9852
✅ 模型已儲存為: raccoon_tanuki_model.pth


In [ ]:
%%writefile app.py
import streamlit as st
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import numpy as np
import os
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

# --- 頁面基本設定 ---
st.set_page_config(page_title="浣熊 vs 狸貓", page_icon="🦝")
st.title("🦝 浣熊 vs 狸貓 AI 辨識器")
st.write("狀態：準備就緒，請上傳圖片。")

# --- 參數設定 ---
MODEL_PATH = 'raccoon_tanuki_model.pth'
CONFIDENCE_THRESHOLD = 0.6

# --- 載入模型 (移除 Cache 以確保穩定性) ---
def get_model():
    device = torch.device("cpu")
    if not os.path.exists(MODEL_PATH):
        return None, None, None

    try:
        checkpoint = torch.load(MODEL_PATH, map_location=device)
        class_names = checkpoint['classes']

        # 建立模型
        model = models.resnet18(weights=None)
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, len(class_names))
        model.load_state_dict(checkpoint['model_state_dict'])
        model.eval()
        return model, class_names, device
    except:
        return None, None, None

# --- 影像處理 ---
def process_image(image):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    return transform(image).unsqueeze(0)

# --- Grad-CAM ---
def get_gradcam(model, input_tensor, original_image):
    target_layers = [model.layer4[-1]]
    cam = GradCAM(model=model, target_layers=target_layers)
    grayscale_cam = cam(input_tensor=input_tensor, targets=None)[0, :]
    img_resized = np.array(original_image.resize((224, 224))) / 255.0
    return show_cam_on_image(img_resized, grayscale_cam, use_rgb=True)

# --- 主程式邏輯 ---
# 1. 先顯示上傳按鈕 (確保這個 UI 永遠存在)
uploaded_file = st.file_uploader("📷 請選擇一張 JPG 或 PNG 圖片", type=["jpg", "jpeg", "png"])

# 2. 如果有上傳，才開始載入模型並預測
if uploaded_file is not None:
    st.write("🔄 正在分析中...")

    # 載入模型
    model, class_names, device = get_model()

    if model is None:
        st.error("找不到模型檔案，請檢查 GitHub 或 Colab 檔案區。")
    else:
        try:
            # 讀圖與預測
            image = Image.open(uploaded_file).convert('RGB')
            input_tensor = process_image(image)

            with torch.no_grad():
                outputs = model(input_tensor)
                probs = torch.nn.functional.softmax(outputs, dim=1)[0]

            top_prob, top_idx = torch.max(probs, 0)
            top_class = class_names[top_idx]
            top_prob_val = top_prob.item()

            # --- 顯示結果區 ---
            st.markdown("---")
            col1, col2 = st.columns(2)

            with col1:
                st.image(image, caption='你的圖片', use_column_width=True)

            with col2:
                st.subheader("📊 分析結果")

                # OOD 判斷
                is_ood = False
                if top_class == 'other':
                    is_ood = True
                    st.error("🚫 結果：以上皆非 (Other)")
                elif top_prob_val < CONFIDENCE_THRESHOLD:
                    is_ood = True
                    st.warning(f"🤔 結果：不確定 (似 {top_class}?)")
                    st.write(f"信心度 {top_prob_val*100:.1f}% 太低。")
                else:
                    if top_class == 'raccoon':
                        st.success("🦝 結果：浣熊 (Raccoon)")
                    elif top_class == 'tanuki':
                        st.info("🍂 結果：狸貓 (Tanuki)")
                    st.metric("信心度", f"{top_prob_val*100:.1f}%")

                st.bar_chart({name: float(p) for name, p in zip(class_names, probs)})

            # --- 進階功能 (熱點圖 + 教學) ---
            if not is_ood:
                st.markdown("---")
                st.subheader("🔥 AI 視覺熱點")
                cam_vis = get_gradcam(model, input_tensor, image)
                st.image(cam_vis, caption='紅色區域為判斷依據', width=350)

                st.markdown("---")
                st.subheader("🎓 特徵比一比")

                # 樣式設定
                style_rac = "border:2px solid #4CAF50; background:#e8f5e9" if top_class == 'raccoon' else "opacity:0.5"
                style_tan = "border:2px solid #FF9800; background:#fff3e0" if top_class == 'tanuki' else "opacity:0.5"

                c1, c2 = st.columns(2)
                with c1:
                    st.markdown(f"""<div style="{style_rac}; padding:10px; border-radius:10px">
                    <h4 style="color:#2E7D32; text-align:center">🦝 浣熊特徵</h4>
                    <ul><li><b>尾巴有環紋</b></li><li>五指分開 (像手)</li><li>眼罩分開</li></ul></div>""", unsafe_allow_html=True)
                with c2:
                    st.markdown(f"""<div style="{style_tan}; padding:10px; border-radius:10px">
                    <h4 style="color:#EF6C00; text-align:center">🍂 狸貓特徵</h4>
                    <ul><li><b>尾巴無環紋</b></li><li>腳掌像狗肉墊</li><li>眼罩相連</li></ul></div>""", unsafe_allow_html=True)

        except Exception as e:
            st.error(f"發生錯誤: {e}")

Overwriting app.py


In [ ]:
import os
import time
import re
import subprocess

# 1. 先殺掉所有舊的程序 (清理環境)
os.system("pkill -f streamlit")
os.system("pkill -f cloudflared")

# 2. 確保 Cloudflare 有安裝 (如果剛剛裝過會跳過)
if not os.path.exists("cloudflared.deb"):
    print("📥 下載連線工具...")
    os.system("curl -L --output cloudflared.deb https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb > /dev/null 2>&1")
    os.system("sudo dpkg -i cloudflared.deb > /dev/null 2>&1")

# 3. 啟動 Streamlit (背景執行)
print("🚀 啟動 Streamlit...")
os.system("streamlit run app.py --server.enableCORS=false --server.enableXsrfProtection=false > /dev/null 2>&1 &")

# 4. 啟動 Cloudflare Tunnel 並將網址寫入 log 檔
print("🔗 正在建立連線 (將輸出寫入 tunnel.log)...")
os.system("nohup cloudflared tunnel --url http://localhost:8501 > tunnel.log 2>&1 &")

# 5. 循環檢查 log 檔，直到抓到網址
print("⏳ 等待網址生成 (約 5-10 秒)...")
found_url = False

for i in range(30): # 最多等 30 秒
    time.sleep(1)
    if os.path.exists("tunnel.log"):
        with open("tunnel.log", "r") as f:
            content = f.read()
            # 尋找 .trycloudflare.com 的網址
            match = re.search(r'https://[a-zA-Z0-9-]+\.trycloudflare\.com', content)
            if match:
                print("\n👇👇👇 成功！請點擊下方連結 (免密碼) 👇👇👇\n")
                print(f"🔗 {match.group(0)}")
                print("\n👆👆👆 點擊這個連結開啟 App 👆👆👆\n")
                found_url = True
                break

if not found_url:
    print("⚠️ 奇怪，還是沒抓到。請在左側檔案區打開 'tunnel.log' 看看裡面有沒有網址。")

🚀 啟動 Streamlit...
🔗 正在建立連線 (將輸出寫入 tunnel.log)...
⏳ 等待網址生成 (約 5-10 秒)...

👇👇👇 成功！請點擊下方連結 (免密碼) 👇👇👇

🔗 https://wishlist-seats-booth-dental.trycloudflare.com

👆👆👆 點擊這個連結開啟 App 👆👆👆

